In [ ]:
import dill

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
base_path = "/home/bryanpu1/projects/aaai_2026/scaling_jax/results"
algo_name = "xland_ad"
run_name = "debug-06-02-25_14_46_07-3c1bc77f-9d04-4c25-8356-67dd1919b82a"

algo_name = "xland_dpt"
run_name = "debug-06-02-25_14_47_19-f8577fb0-3fb8-4664-8982-64d5985a9961"

algo_name = "xland_expi"
run_name = "debug-06-02-25_14_47_55-e68410db-d131-44d5-8720-a660d26eb3f8"

algo_name = "bandit_ad"
run_name = "debug-06-05-25_13_19_10-d1262807-a575-490d-abc8-35bae6930c8d"

In [ ]:
data = dill.load(open(f"{base_path}/{algo_name}/{run_name}/eval_info.dill", "rb"))

In [ ]:
data.keys()

In [ ]:
data["action_counts"]

In [ ]:
data["env_params"].shape

In [ ]:
if algo_name.startswith("xland"):
    for k, v in data.items():
        plt.figure()
        plt.plot(range(len(v)), v, label=k)
        plt.title(f"Distribution of returns for {k}")
        plt.xlabel("Episode")
        plt.ylabel("Return")
        plt.show()
elif algo_name.startswith("bandit"):
    (num_envs, num_eps) = data["episode_lengths"].shape
    xrange = range(num_eps)

    num_cols = 5
    num_rows = int(np.ceil(num_envs / 5))

    fig, axes = plt.subplots(
        num_rows,
        num_cols,
        figsize=(5 * num_cols, 5 * num_rows),
        layout="constrained",
    )

    axes = axes.flatten()

    for env_i in range(num_envs):
        axes[env_i].set_title(f"Regret for Env {env_i}")
        axes[env_i].set_xlabel("Rounds")
        axes[env_i].set_ylabel("Regret")

        for task_i in range(int(np.ceil(num_eps / data["switch_freq"]))):
            opt = (np.arange(data["switch_freq"]) + 1) * np.max(data["env_params"][env_i, task_i], axis=-1)
            rews = np.cumsum(
                data["episode_returns"][env_i, task_i * data["switch_freq"]:(task_i + 1) * data["switch_freq"]],
                axis=-1,
            )
            regret = opt - rews
            axes[env_i].plot(task_i * data["switch_freq"] + np.arange(data["switch_freq"]), regret)

            axes[env_i].axvline(
                x=task_i * data["switch_freq"],
                label=f"Task i" if task_i == 0 and env_i == 0 else "",
                linestyle="--",
                color="black"
            )

    fig.legend(
        bbox_to_anchor=(0.0, 1.0, 1.0, 0.0),
        loc="lower center",
        ncols=5,
        borderaxespad=0.0,
        frameon=True,
        fontsize="8", 
    )
    fig.show()